In [1]:
import sys
import os
import pandas as pd
from datetime import datetime

print("当前工作目录:", os.getcwd())

projet_path = os.path.abspath('..')  
print(projet_path)
if projet_path not in sys.path:
    sys.path.append(projet_path)

当前工作目录: d:\Edu\SelfPresentation_Multimodal_airbnb\sanboxJO
d:\Edu\SelfPresentation_Multimodal_airbnb


# test on paris:

In [2]:
df6=pd.read_csv("data_jo\listings_paris2406.csv")
print("jo paris 06:",len(df6))

df9=pd.read_csv("data_jo\listings_paris2409.csv")
print("jo paris 09:", len(df9))


df6['scraped_date'] = '2024Q2'
df9['scraped_date'] = '2024Q3'
dfparis = pd.concat([df6, df9], ignore_index=True)
print(len(dfparis))

# dfparis=pd.read_csv("data_jo_processed/listings_jo_paris.csv")
# print(len(dfparis))

jo paris 06: 95885
jo paris 09: 95461
191346


In [138]:
#描述房东数量变化：
hosts_in = set(df9['host_id']) - set(df6['host_id']) # 9月有，但6月没有
hosts_out = set(df6['host_id']) - set(df9['host_id']) #6月有，但9月没有
hosts_io= set(df9['host_id']) ^ set(df6['host_id']) # 属于 6月 或 9月，但不同时属于两者;所有变化的房东（新增 + 消失）
print(f'new hosts after JO : {len(hosts_in)}({len(hosts_in)*100/len(df9):.2f}%)')
print(f'hosts dispeared after JO : {len(hosts_out)}({len(hosts_out)*100/len(df9):.2f}%)')
print(f"host changed during JO :{len(hosts_io)}")


new hosts after JO : 5897(6.18%)
hosts dispeared after JO : 7024(7.36%)
host changed during JO :12921


In [139]:
#描述房源数量变化
hosts_in = set(df9['id']) - set(df6['id']) # 9月有，但6月没有
hosts_out = set(df6['id']) - set(df9['id']) #6月有，但9月没有
hosts_io= set(df9['id']) ^ set(df6['id']) # 属于 6月 或 9月，但不同时属于两者;所有变化的房东（新增 + 消失）
print(f'new listings after JO : {len(hosts_in)}({len(hosts_in)*100/len(df9):.2f}%)')
print(f'listings dispeared after JO : {len(hosts_out)}({len(hosts_out)*100/len(df9):.2f}%)')
print(f"listings changed during JO :{len(hosts_io)}")


new listings after JO : 9882(10.35%)
listings dispeared after JO : 10306(10.80%)
listings changed during JO :20188


In [8]:
#增加status列：
# 统计每个房东在每个时期的房源数量
hosts6 = df6.groupby('host_id').size().reset_index(name='listings_06')
hosts9 = df9.groupby('host_id').size().reset_index(name='listings_09')

# 合并两个时期
merged = hosts6.merge(hosts9, on='host_id', how='outer')

# 分类:新房东，消失房东，listings增加/减少/不变
merged['status'] = merged.apply(
    lambda r:
        'new_host' if pd.isna(r['listings_06']) else
        'disappeared_host' if pd.isna(r['listings_09']) else
        ('expanded' if r['listings_09'] > r['listings_06'] else
         'reduced' if r['listings_09'] < r['listings_06'] else
         'no_change'),
    axis=1
)

print(merged.head())
print(len(merged))
print(merged.status.value_counts())

dfparis=dfparis.merge(merged[['host_id','status']], left_on='host_id', right_on='host_id',how='left')
print(len(dfparis))
display(dfparis.head())

   host_id  listings_06  listings_09     status
0     2626          3.0          1.0    reduced
1     2883          NaN          2.0   new_host
2     3631          1.0          1.0  no_change
3     5468          1.0          1.0  no_change
4     6792          8.0         11.0   expanded
77876
status
no_change           63061
disappeared_host     7024
new_host             5897
reduced               948
expanded              946
Name: count, dtype: int64
191346


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,scraped_date,host_about_q2,host_about_changed,host_picture_url_q2,host_picture_url_changed,status
0,80260,https://www.airbnb.com/rooms/80260,20240610195007,2024-06-13,previous scrape,Nice studio in Jourdain's village,NaN,NaN,https://a0.muscache.com/pictures/716553/6c14f2...,333548,...,1,0,0,1.29,2024Q2,"My name is Charlotte, I'm 26 and I study cinem...",NaN,https://a0.muscache.com/im/users/333548/profil...,NaN,no_change
1,3109,https://www.airbnb.com/rooms/3109,20240610195007,2024-06-12,city scrape,zen and calm,Lovely Appartment with one bedroom with a Quee...,Good restaurants<br />very close the Montparna...,https://a0.muscache.com/pictures/miso/Hosting-...,3631,...,1,0,0,0.05,2024Q2,NaN,NaN,https://a0.muscache.com/im/users/3631/profile_...,NaN,no_change
2,80301,https://www.airbnb.com/rooms/80301,20240610195007,2024-06-13,city scrape,toits de Paris,"On the top,sharing my space,you have your bedr...","SAFE neighborhood,late bus and subway,velibe(b...",https://a0.muscache.com/pictures/742354/2e66b9...,433758,...,1,1,0,0.27,2024Q2,"Doctorat Sorbonne 1,Litterature sciences huma...",NaN,https://a0.muscache.com/im/users/433758/profil...,NaN,no_change
3,5396,https://www.airbnb.com/rooms/5396,20240610195007,2024-06-13,city scrape,Your perfect Paris studio on Île Saint-Louis,"NEW SOFA-BED SINCE JUNE 2023, Please disregard...","You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,...,1,0,0,2.21,2024Q2,We have spent a lot of time traveling for work...,NaN,https://a0.muscache.com/im/users/7903/profile_...,NaN,no_change
4,7397,https://www.airbnb.com/rooms/7397,20240610195007,2024-06-13,city scrape,MARAIS - 2ROOMS APT - 2/4 PEOPLE,"VERY CONVENIENT, WITH THE BEST LOCATION !",NaN,https://a0.muscache.com/pictures/67928287/330b...,2626,...,3,0,0,2.21,2024Q2,"I am a writer,54, author of novels, books of l...",NaN,https://a0.muscache.com/im/pictures/user/ad6a9...,NaN,reduced


In [3]:
import pandas as pd
import numpy as np


if 'host_about_q2' not in dfparis:
    df_q2 = dfparis[dfparis['scraped_date'] == '2024Q2'][['host_id', 'id', 'host_about']]
    df_q2 = df_q2.rename(columns={'host_about': 'host_about_q2'})
    dfparis = dfparis.merge(df_q2, on=['host_id', 'id'], how='left')


# 计算文本相似度（使用 difflib.SequenceMatcher）
from difflib import SequenceMatcher
def text_similarity(a, b):
    if pd.isna(a) or pd.isna(b):
        return None
    return SequenceMatcher(None, str(a), str(b)).ratio()

# 标记 Q3 相对于 Q2 的变化
def host_about_change(row, threshold=0.85):
    """
    没改动/例外都被填为nan （包括所有Q2房东）
    新房东/新文本/新bio填1或sim
    """
    if row['scraped_date'] != '2024Q3':
        return None  # 只对 Q3 标记
    if pd.isna(row['host_about_q2']) and pd.notna(row['host_about']):
        return 1  # Q2 没有文本，Q3 新增文本或新房东
    if pd.notna(row['host_about_q2']) and pd.notna(row['host_about']):#老房东&bio不为空# row['host_about'] != row['host_about_q2']:
        sim= text_similarity(row['host_about_q2'], row['host_about']) 
        if sim < threshold:## 老文本有明显变化
            return sim 
        else:
            return None#无明显改变是就填NAN
    return None  # 其他情况

if "host_about_change" not in dfparis:
    dfparis['host_about_changed'] = dfparis.apply(host_about_change, axis=1)

# 查看 Q3 中变化统计：没改动都被填为nan，新房东/新文本/新bio填1或sim:
print(dfparis.host_about_changed.notna().value_counts())
print(dfparis.host_about_changed.value_counts())


#有5k明显改变，其中4k是房东或者新添加文本

host_about_changed
False    186108
True       5238
Name: count, dtype: int64
host_about_changed
1.000000    4275
0.175573     336
0.020240     146
0.089844      33
0.289762      28
            ... 
0.014634       1
0.218810       1
0.785083       1
0.318408       1
0.818182       1
Name: count, Length: 274, dtype: int64


In [4]:
if "host_picture_url_q2" not in dfparis:
    df_q2 = dfparis[dfparis['scraped_date'] == '2024Q2'][['host_id', 'id', 'host_picture_url']]
    df_q2 = df_q2.rename(columns={'host_picture_url': 'host_picture_url_q2'})
    dfparis = dfparis.merge(df_q2, on=['host_id', 'id'], how='left')


# 标记 Q3 相对于 Q2 的变化
def host_picture_url_change(row):
    if row['scraped_date'] != '2024Q3':
        return None # 只对 Q3 标记
    if pd.isna(row['host_picture_url_q2']) and pd.notna(row['host_picture_url']):
        return 1  # Q2 没有文本，Q3 新增文本
    if pd.notna(row['host_picture_url_q2']) and pd.notna(row['host_picture_url']) and row['host_picture_url'] != row['host_picture_url_q2']:
        return 1  # 老文本被修改
    return None  # 其他情况

if "host_picture_url_changed" not in dfparis:
    dfparis['host_picture_url_changed'] = dfparis.apply(host_picture_url_change, axis=1)

# 查看 Q3 变化统计
print(dfparis.host_picture_url_changed.notna().value_counts())
print(dfparis.host_picture_url_changed.value_counts())

#12K房东修改了照片

host_picture_url_changed
False    178811
True      12535
Name: count, dtype: int64
host_picture_url_changed
1.0    12535
Name: count, dtype: int64


In [9]:
display(dfparis.head())
dfparis.to_csv("data_jo_processed/listings_jo_paris.csv",index=False)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,scraped_date,host_about_q2,host_about_changed,host_picture_url_q2,host_picture_url_changed,status
0,80260,https://www.airbnb.com/rooms/80260,20240610195007,2024-06-13,previous scrape,Nice studio in Jourdain's village,NaN,NaN,https://a0.muscache.com/pictures/716553/6c14f2...,333548,...,1,0,0,1.29,2024Q2,"My name is Charlotte, I'm 26 and I study cinem...",NaN,https://a0.muscache.com/im/users/333548/profil...,NaN,no_change
1,3109,https://www.airbnb.com/rooms/3109,20240610195007,2024-06-12,city scrape,zen and calm,Lovely Appartment with one bedroom with a Quee...,Good restaurants<br />very close the Montparna...,https://a0.muscache.com/pictures/miso/Hosting-...,3631,...,1,0,0,0.05,2024Q2,NaN,NaN,https://a0.muscache.com/im/users/3631/profile_...,NaN,no_change
2,80301,https://www.airbnb.com/rooms/80301,20240610195007,2024-06-13,city scrape,toits de Paris,"On the top,sharing my space,you have your bedr...","SAFE neighborhood,late bus and subway,velibe(b...",https://a0.muscache.com/pictures/742354/2e66b9...,433758,...,1,1,0,0.27,2024Q2,"Doctorat Sorbonne 1,Litterature sciences huma...",NaN,https://a0.muscache.com/im/users/433758/profil...,NaN,no_change
3,5396,https://www.airbnb.com/rooms/5396,20240610195007,2024-06-13,city scrape,Your perfect Paris studio on Île Saint-Louis,"NEW SOFA-BED SINCE JUNE 2023, Please disregard...","You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,...,1,0,0,2.21,2024Q2,We have spent a lot of time traveling for work...,NaN,https://a0.muscache.com/im/users/7903/profile_...,NaN,no_change
4,7397,https://www.airbnb.com/rooms/7397,20240610195007,2024-06-13,city scrape,MARAIS - 2ROOMS APT - 2/4 PEOPLE,"VERY CONVENIENT, WITH THE BEST LOCATION !",NaN,https://a0.muscache.com/pictures/67928287/330b...,2626,...,3,0,0,2.21,2024Q2,"I am a writer,54, author of novels, books of l...",NaN,https://a0.muscache.com/im/pictures/user/ad6a9...,NaN,reduced


In [10]:
df_change=dfparis[(dfparis['status']=='new_host')|(dfparis['status']=='expanded')|(dfparis['host_about_changed']==1)|(dfparis['host_picture_url_changed']==1)]
df_stable = dfparis[~dfparis['host_id'].isin(df_change['host_id'])]#.isin() 来做“是否在某个列表中”的列级比较，然后加 ~ 表示取反。
print(df_change.host_about.notna().value_counts())
print(df_stable.host_about.notna().value_counts())
print(f"len change: {len(df_change)}")
print(f"len stable: {len(df_stable)}")

host_about
True     16198
False    14156
Name: count, dtype: int64
host_about
False    90445
True     58638
Name: count, dtype: int64
len change: 30354
len stable: 149083


In [151]:
print(df_change.host_about.notna().value_counts())
print(df_stable.host_about.notna().value_counts())

host_about
True     16198
False    14156
Name: count, dtype: int64
host_about
False    90445
True     58638
Name: count, dtype: int64


# synthèse de change :

In [2]:
#修改函数但是仍然用sys.modules 中缓存的，没有显示更新
import importlib
from utils import preprocess_data
importlib.reload(preprocess_data)

from utils.preprocess_data import split_change_stable,save_global_change_stable_csv


# jo实验组：paris2403

In [3]:
df3=pd.read_csv("data_jo\listings_paris2403.csv")
df6=pd.read_csv("data_jo\listings_paris2406.csv")
dfparis2024, dfparis2024_change, dfparis2024_stable=split_change_stable(df3, df6, year='2024')

===================================INPUT DATA===================================
len df MARS: 84397
len df JUIN: 95885
len df total : 180282 

===================================APRCU GLOBAL===================================
※ HOST STAUTS CHANGE (Q2): 
 status
old_host         81141
reactive_host    11957
new_host          2787
Name: count, dtype: int64 

LISTINGS CHANGE:
new listings after JO : 19299(20.13%)
listings dispeared after JO : 7811(8.15%)
listings changed during JO :27110

HOST ABOUT CHANGE (change:1, no_change:0) :
host_about_changed
0    171443
1      8839
Name: count, dtype: int64 

HOST PICTURE CHANGE (change:1, no_change:0):
host_picture_url_changed
True    180282
Name: count, dtype: int64
host_picture_url_changed
0    157298
1     22984
Name: count, dtype: int64 

HOST IM global (no_change:0, 1change :1, 2change:2): 
 presentation_change
0    72128
1    15691
2     8066
Name: count, dtype: int64 

===================================SUMMARY ON CHANGE==================

d:\Edu\SelfPresentation_Multimodal_airbnb\utils\preprocess_data.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfparisQ2['presentation_change'] = dfparisQ2.apply(presentation_change_level, axis=1)
d:\Edu\SelfPresentation_Multimodal_airbnb\utils\preprocess_data.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfparisQ2['is_changed'] = (


In [5]:
# dfparis2024.to_csv("data_jo_processed\listings_jo_paris2024.csv", index=False)
# dfparis2024_change.to_csv("data_jo_processed\listings_jo_paris2024_change.csv", index=False)
# dfparis2024_stable.to_csv("data_jo_processed\listings_jo_paris2024_stable.csv", index=False)
save_global_change_stable_csv([dfparis2024, dfparis2024_change, dfparis2024_stable], year='2024', location='paris', output_folder="data_jo_grouped")


NB. csv files enter strictly in order : global-change-stable!
default output folder : data_jo_grouped
✔ csv saved in data_jo_grouped\listings_jo_paris2024.csv!
✔ csv saved in data_jo_grouped\listings_jo_paris2024_change.csv!
✔ csv saved in data_jo_grouped\listings_jo_paris2024_stable.csv!


## 同期对照组：london2403:

In [6]:
dfl3=pd.read_csv("data_jo\listings_london2403.csv")
dfl6=pd.read_csv("data_jo\listings_london2406.csv")
dflondon, dflondon_change, dflondon_stable=split_change_stable(dfl3, dfl6)


===================================INPUT DATA===================================
len df MARS: 90852
len df JUIN: 93481
len df total : 184333 

===================================APRCU GLOBAL===================================
※ HOST STAUTS CHANGE (Q2): 
 status
old_host         86357
reactive_host     5423
new_host          1701
Name: count, dtype: int64 

LISTINGS CHANGE:
new listings after JO : 14777(15.81%)
listings dispeared after JO : 12148(13.00%)
listings changed during JO :26925

HOST ABOUT CHANGE (change:1, no_change:0) :
host_about_changed
0    175830
1      8503
Name: count, dtype: int64 

HOST PICTURE CHANGE (change:1, no_change:0):
host_picture_url_changed
True    184333
Name: count, dtype: int64
host_picture_url_changed
0    167511
1     16822
Name: count, dtype: int64 

HOST IM global (no_change:0, 1change :1, 2change:2): 
 presentation_change
0    75727
1    10183
2     7571
Name: count, dtype: int64 

===================================SUMMARY ON CHANGE================

d:\Edu\SelfPresentation_Multimodal_airbnb\utils\preprocess_data.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfparisQ2['presentation_change'] = dfparisQ2.apply(presentation_change_level, axis=1)
d:\Edu\SelfPresentation_Multimodal_airbnb\utils\preprocess_data.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfparisQ2['is_changed'] = (


In [7]:
save_global_change_stable_csv([dflondon, dflondon_change, dflondon_stable], year='2024', location='london', output_folder="data_jo_grouped")


NB. csv files enter strictly in order : global-change-stable!
default output folder : data_jo_grouped
✔ csv saved in data_jo_grouped\listings_jo_london2024.csv!
✔ csv saved in data_jo_grouped\listings_jo_london2024_change.csv!
✔ csv saved in data_jo_grouped\listings_jo_london2024_stable.csv!


## 同期对照组：paris2025 3-6

In [70]:
df3=pd.read_csv("data_jo\listings_paris2503.csv")
df6=pd.read_csv("data_jo\listings_paris2506.csv")
dfparis2025, dfparis2025_change, dfparis2025_stable=split_change_stable(df3, df6,year='2025')

len df MARS: 86064
len df JUIN: 84055
len df total : 170119 

※ HOST STAUTS CHANGE (Q2): 
 status
old_host         80485
reactive_host     3384
new_host           186
Name: count, dtype: int64 

LISTINGS CHANGE:
new listings after JO : 6142(7.31%)
listings dispeared after JO : 8151(9.70%)
listings changed during JO :14293

HOST ABOUT CHANGE :
host_about_changed
True    170119
Name: count, dtype: int64
host_about_changed
0    165807
1      4312
Name: count, dtype: int64 

HOST PICTURE CHANGE :
host_picture_url_changed
True    170119
Name: count, dtype: int64
host_picture_url_changed
0    161644
1      8475
Name: count, dtype: int64 



d:\Edu\SelfPresentation_Multimodal_airbnb\utils\preprocess_data.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\Edu\SelfPresentation_Multimodal_airbnb\utils\preprocess_data.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df9['scraped_date'] = f"{year}Q2"


HOST IM global : presentation_change
0    74412
1     6499
2     3144
Name: count, dtype: int64 

※ len change: 9643
※ len stable: 74412 

※ HOST STATUS CHANGE:
status
old_host         0.629783
reactive_host    0.350928
new_host         0.019289
Name: count, dtype: float64
※ HOST IM CHANGE:
presentation_change
1    0.67396
2    0.32604
Name: count, dtype: float64
※ HOST IM CHANGE (exclude no_change):
 host_about_changed  host_picture_url_changed
0                   1                           5331
1                   1                           3144
                    0                           1168
Name: count, dtype: int64

Temps d'exécution :27.14 sec.


In [66]:
dfparis2025_change['host_about'].notna().value_counts(dropna=False)

host_about
True     5148
False    4495
Name: count, dtype: int64

In [71]:
dfparis2025.to_csv("data_jo_processed\listings_jo_paris2025.csv", index=False)
dfparis2025_change.to_csv("data_jo_processed\listings_jo_paris2025_change.csv", index=False)
dfparis2025_stable.to_csv("data_jo_processed\listings_jo_paris2025_stable.csv", index=False)